# Query Rewrite and Web Search Context for LLMs

- This code snippet demonstrates how to rewrite a query, perform a web search, and prepare the results for use with a language model (LLM). It includes functions to rewrite the query using a language model, perform a web search using the rewritten query, and format the search results for the LLM.





In [1]:
import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import load_prompt
import sys
import logging
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search
from IPython.display import Markdown, display
from datetime import datetime
import time
import os
import pytz
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [ ]:
# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2025-03-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):    
        
        # TODO Customize your prompt
        QUERY_REWRITE_PROMPT = """
          You are an expert in rewriting user queries to improve search results. 
          Here is the user's original query:
          {user_query}
          
          <<Format>>
          The response should be in JSON Object format with the following keys
          {{
          "search_query": "검색용 재작성된 질문"
          }}
        """
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
                  user_query=query)},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )

        print("Rewritten query for search:", response.choices[0].message.content.strip())
        
        return json.loads(response.choices[0].message.content.strip())


In [ ]:

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_web_search(RESULTS_COUNT, input, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()

    print(f"############## Web Search Mode: {web_search_mode}")
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input, client)

    # 웹 검색 실행 (비동기)
    print("Search rewrited search queries...")   
    results = await web_search(query_rewrite, RESULTS_COUNT, web_search_mode=web_search_mode, product_name=product_name)

    # Create a prompt for generating updated, targeted

    GENERATE_PROMPT = """
    <<Customize your prompt here>>
    You are an expert in generating responses based on search results. Your task is to generate a response using the provided search results and the user's query.
    Here are the search results:
    {contexts}
    Here is the user's query:
    {user_query}
    """
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=None,
            contexts=results if results else '검색 결과 없음',
            user_query=query_rewrite,
        )},
        {"role": "user", "content": query_rewrite}
    ]
            
    print("Generate response...")   
    
    if web_search_mode == "google" and isinstance(results, list) and results:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=answer_messages,
            top_p=0.9,
            max_tokens=1500
        )
        display(Markdown(response.choices[0].message.content))
    elif web_search_mode == "bing" and isinstance(results, list) and results:
        for result in results:
            if isinstance(result, str):
                display(Markdown(result))
            else:
                display(Markdown(str(result)))

    else:
        display(Markdown("검색 결과가 없습니다. 다른 질문을 해주세요."))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [16]:
RESULTS_COUNT = 5

inputs = [
    "엄빠폰 느낌좋은걸로 추구미 비교 추천",
    # "내 동생을 위한 스마트폰 추천",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "테슬라 모델3와 모델 Y 차이점이 뭐야",
    # "내 핸드폰 요금제를 절약하는 법",
]

web_search_mode = "bing"

for input in inputs:
    await process_web_search(RESULTS_COUNT, input, web_search_mode, product_name="ㅇㅇ전자")  # product_name은 필요에 따라 변경 가능

Original Input: 엄빠폰 느낌좋은걸로 추구미 비교 추천
############## Web Search Mode: bing


Rewritten query for search: {
  "search_query": "엄마 아빠 스마트폰 추천 및 비교 - 느낌 좋은 모델 찾기"
}
Search rewrited search queries...
Generate response...


부모님을 위한 스마트폰 추천 및 비교에 대한 최신 정보는 다음과 같습니다. 2025년형 스마트폰 모델을 기준으로, 사용의 용이성, 가격 대비 성능, 배터리 수명 등을 고려하여 추천된 몇 가지 모델을 소개합니다.

### 추천 스마트폰 모델

1. **삼성 갤럭시 A34**
   - **특징**: 가성비가 뛰어나고 직관적인 UI.
   - **가격**: 최저가 구매링크 확인 필요.

2. **아이폰 SE 3세대**
   - **특징**: 아이폰 특유의 안정성과 간편한 사용.
   - **가격**: 최저가 구매링크 확인 필요.

3. **LG Q92** (중고 또는 재고)
   - **특징**: 기본적인 성능과 가성비가 우수.
   - **가격**: 최저가 구매링크 확인 필요.

4. **샤오미 레드미노트 12**
   - **특징**: 성능과 가격 모두 만족스러운 모델.
   - **가격**: 해외직구 가능, 최저가 구매링크 확인 필요.

5. **삼성 갤럭시 와이드6**
   - **특징**: 효도폰 베스트셀러로, 사용법이 간단해 적합.
   - **가격**: 최저가 구매링크 확인 필요.

### 추천 이유 및 사용 팁
부모님들은 통상적으로 고사양의 스마트폰보다는 사용이 간편하고 가격 대비 성능이 우수한 모델을 선호합니다. 특히, 카카오톡과 같은 기본적인 앱 사용이 원활하도록 데이터를 백업해 두는 것이 좋습니다. 

또한, 가성비가 뛰어난 제품들이 많으니 구매 전 가격 비교를 통해 최저가를 찾아보세요! 

더 많은 정보는 다음 링크를 통해 확인할 수 있습니다:
- [부모님을 위한 스마트폰 추천 리스트 (2025년형)](https://okaminer.com/%EB%B6%80%EB%AA%A8%EB%8B%98%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8F%B0-%EC%B6%94%EC%B2%9C-%EB%A6%AC%EC%8A%A4%ED%8A%B8-2025%EB%85%84%ED%98%95/)【3:0†source】
- [50,60대 부모님을 위한 스마트폰 추천 TOP5 (2025년 최신)](https://hazzistory.com/entry/5060%EB%8C%80-%EB%B6%80%EB%AA%A8%EB%8B%98%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8F%B0-%EC%B6%94%EC%B2%9C-TOP5-2025%EB%85%84-%EC%B5%9C%EC%8B%A0)【3:1†source】.

elapsed time: 28.15 seconds




# Challenge: Rewrite the query to be more specific and perform a web search

### Requirements:
1. Please only show our company's products.
1. Please provide detailed answers even short questions.